![spotify_logo](../img/spotify_logo.png)

# Spotify REST API Challenge

__What to listen?__

Create your own playlist based on the related artists and their most popular tracks using the [Spotify REST API](https://developer.spotify.com/)

In [ ]:
# Imports
import pandas as pd
import requests
from dotenv import dotenv_values

### Get access!!!

Get your `client_id` and `client_secret` to generate your __token__ access

In [ ]:
config = dotenv_values('.env')
#config

In [ ]:
# Generate token with a POST request
client_id = config.get('CLIENT_ID')   # CLIENT ID
client_secret = config['CLIENT_SECRET']   # CLIENT SECRET
auth_url = 'https://accounts.spotify.com/api/token'

In [ ]:
auth_response = requests.post(auth_url, {'grant_type': 'client_credentials',
                                         'client_id': client_id,
                                         'client_secret': client_secret}).json()

In [ ]:
access_token = auth_response['access_token']
#auth_response

### Set your main variables!!!

Set the `base_uri` (i.e.: end-point), parameters and `headers` for your __GET__ operations

In [ ]:
# Base end-point construction

base_url = 'https://api.spotify.com/v1/'
resource = 'artists/'

header_info = {'Authorization': 'Bearer {token}'.format(token=access_token)}
#header_info

### Create your new playlist!!!

Use [`/related-artists`](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-an-artists-related-artists) and [`/top-tracks`](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-an-artists-top-tracks) in order to get the data that you need.

__Here you have the different artists to start with:__

- __Option 1:__ 0L8ExT028jH3ddEcZwqJJ5

- __Option 2:__ 4Y7tXHSEejGu1vQ9bwDdXW

- __Option 3:__ 6FBDaR13swtiWwGhX1WQsP

- __Option 4:__ 0kyQwKHCZnKE7kTXkxXjrB

- __Option 5:__ 2d0hyoQ5ynDBnkvAbJKORj

- __Option 6:__ 3bgsNtcf5d5h9jbQbohfBK

- __Option 7:__ 7mWCSSOYqm4E9mB7V4ot6S

- __Option 8:__ 64KEffDW9EtZ1y2vBYgq8T

- __Option 9:__ 4k1ELeJKT1ISyDv8JivPpB

- __Option 10:__ 4Z8W4fKeB5YxbusRsdQVPb

- __Option 11:__ 26dSoYclwsYLMAKD3tpOr4

- __Option 12:__ 7y97mc3bZRFXzT2szRM4L4

- __Option 13:__ 1w5Kfo2jwwIPruYS2UWh56


> Remember to check the [Requests](https://requests.readthedocs.io/en/latest/) library docs!!!

---

### END-POINT Nº 1 - EXTRACT RELATED ARTIST FROM 'artist_id'

In [ ]:
# Here you can complete your endpoint URI
#END-POINT Nº 1 - EXTRACT RELATED ARTIST FROM 'artist_id'
#Fragments to compose the endpoint
base_url = 'https://api.spotify.com/v1/'
resource = 'artists/'
artist_id = '7y97mc3bZRFXzT2szRM4L4'
method = '/related-artists'

#End-points one
full_endpoint = base_url + resource + artist_id + method
full_endpoint

In [ ]:
# Start building your playlist!!!
related_artists_response = requests.get(full_endpoint, headers = header_info)   #Get to spotify
related_artists_response   #Response of the get - 200 means that everything is correct

In [ ]:
related_artists_response.content   #This data is byte type, as it's possible to see in the first character (letter b)

In [ ]:
related_artists_json = related_artists_response.json()   #Transform the data to json
related_artists_json.keys()   #Print the json keys. It's possible because in the json there is a dictionary
related_artists_json['artists']   #Print related artist of id artist
len(related_artists_json['artists'])   #Len of related artist list
related_artists_json['artists'][0]['name']   #Extract the name of first related artist

#Whit this list comprehension it's possible to obtain the names of all related artist
[related_artists_json['artists'][i]['name'] for i in range(len(related_artists_json['artists']))]

In [ ]:
#Transform this json in a dataframe and store only this columns: 'id', 'name', 'genres', 'popularity', 'followers'
df_related_artists = pd.DataFrame(related_artists_json['artists'])[['id', 'name', 'genres', 'popularity', 'followers']]
#Rename the column names
df_related_artists = df_related_artists.rename(columns = {'id': 'related_id',
                                                          'name': 'related_name',
                                                          'genres': 'related_genres',
                                                          'popularity': 'related_popularity',
                                                          'followers': 'related_followers'})
#Create a new column with 'artist_id'
df_related_artists['artist_id'] = artist_id

#The 'followers' column is a dictionary type, so whit this lambda function, it's possible to extract the interesting data 
#of this column aand replaces this extracted column with the previous one.
df_related_artists['related_followers'] = df_related_artists['related_followers'].apply(lambda row: row['total'])
df_related_artists.head()

### END-POINT Nº2 - EXTRACT THE TOP-TRACKS OF THE 'artist_id'

In [ ]:
#END-POINT Nº2 - EXTRACT THE TOP-TRACKS OF THE 'artist_id'
#Fragments to compose the endpoint
base_url = 'https://api.spotify.com/v1/'
resource = 'artists/'
artist_id = '7y97mc3bZRFXzT2szRM4L4'
method2 = '/top-tracks'
argument = '?market=ES'

#End-point 2
full_endpoint = base_url + resource + artist_id + method2 + argument
full_endpoint

In [ ]:
#NOTE: it's possible to write '.json()' at the end of the get and then the response is already a json directly.
top_tracks_response = requests.get(full_endpoint, headers = header_info)   #Get to Spotify
top_tracks_response.status_code   #Status of the get - 200 means that everything is correct

In [ ]:
top_tracks_json = top_tracks_response.json()   #Transform the data to json
top_tracks_json.keys()  #Print the json keys.
#top_tracks_json.values()

In [ ]:
#Whit this list comprehension it's possible to obtain the names of all top-tracks of 'artist_id'
[top_tracks_json['tracks'][i]['name'] for i in range(len(top_tracks_json['tracks']))]

In [ ]:
#NOTE: 'uri' is the unique identifier of a track
#Transform this data in a dataframe, includinn only the 'id', 'href', 'nam'e and 'uri' columns of each track. 
df_tracks = pd.DataFrame(top_tracks_json['tracks'])[['id', 'href','name','uri']]
df_tracks
df_tracks.info()

In [ ]:
#Function 'get_top_tracks' to obtain the top-tracks of one artist. This function includes the codes of cell above. 
#Input: artist_id | output: top-tracks dataframe of this artist.
def get_top_tracks(artist_id):
    #Fragments to compose the endpoint.
    base_url = 'https://api.spotify.com/v1/'
    resource = 'artists/'
    method2 = '/top-tracks'
    argument = '?market=ES'
    #End-point
    full_endpoint = base_url + resource + artist_id + method2 + argument  
    #Get info to Spotify.
    top_tracks_response = requests.get(full_endpoint, headers = header_info).json()
    
    #Create a in a dataframe with this data, including only the 'id', 'href', 'nam'e and 'uri' columns of each track.
    df_tracks = pd.DataFrame(top_tracks_response['tracks'])[['id', 'href','name','uri']]
    #Change the names of dataframe columns.
    df_tracks = df_tracks.rename(columns = {'id': 'related_id',
                                            'href': 'related_href',
                                            'name': 'related_track_name',
                                            'uri': 'related_uri'})
    
    #Include a new colum with the id_related_artist.
    df_tracks['related_id'] = artist_id
    #Include a new colum with the name_related_artist. This name is also extrated to the json.
    df_tracks['related_artist_name'] = top_tracks_response['tracks'][0]['album']['artists'][0]['name']
    
    return df_tracks
    

In [ ]:
#Execute the function
artist_id = '7y97mc3bZRFXzT2szRM4L4'
df_tracks = get_top_tracks(artist_id)
df_tracks.head()

### END-POINT Nº3 - EXTRACT THE INFORMATION OF THE 'artist_id'

In [ ]:
#END-POINT Nº3 - EXTRACT THE INFORMATION OF THE 'artist_id'
#Whit this code, it's possible to extract the data related to 'artist_id'.
#Fragments to compose the endpoint.
base_url = 'https://api.spotify.com/v1/'
id_selected = '7y97mc3bZRFXzT2szRM4L4'
resource = 'artists'
parameters = f'/{id_selected}'
#End-point.
url = base_url + resource + parameters
print(url)
#Get function
response = requests.get(url, headers=header_info).json()

#Extract data from json with this list comprehension. Store this data columns in a dictionary.
dict_artist = dict((key, [response[key]]) for key in ['id', 'name', 'genres', 'popularity', 'followers'] if key in response)
#Transform the dictionary in dataframe.
df_artist = pd.DataFrame(dict_artist)
#Change the names of dataframe columns.
df_artist = df_artist.rename(columns = {'id': 'artist_id',
                                        'name': 'artist_name',
                                        'genres': 'artist_genres',
                                        'popularity': 'artist_popularity',
                                        'followers': 'artist_followers'})

#The 'followers' column is a dictionary type, so whit this lambda function, it's possible to extract the interesting data 
#of this column aand replaces this extracted column with the previous one.
df_artist['artist_followers'] = df_artist['artist_followers'].apply(lambda row: row['total'])
#df_artist.drop('artist_followers', axis=1, inplace=True)
df_artist

### MERGE RELATED ARTISTS WITH ARTIST

In [ ]:
#Merge 'related_artists' with 'artist' dataframe using 'artist_id'
df_related_artists = pd.merge(df_related_artists, df_artist, on='artist_id')
df_related_artists.head()

### END-POINT Nº4 - EXTRACT THE TOP-TRACKS OF THE EACH RELATED ARTIST

In [ ]:
df_all_tracks = pd.DataFrame()   #Create empty dataframe

#In this loop, extract each artist id from the related artists dataframe obtained above. And introduce this each artist id
#in the 'get_top_tracks' function. This function, described above, returns a dataframe with top-tracks of this artist. At
#the end
for each_artist_id in df_related_artists['related_id']:
    each_df_tracks = get_top_tracks(each_artist_id)
    df_all_tracks = pd.concat([df_all_tracks, each_df_tracks], axis=0)
    
df_all_tracks.head()
#df_tracks['related_id'] = id_artist

In [ ]:
#Merge previous dataframe with the information of artist and related artists with 'top-track' dataframe using 'related_id'
df_related_artists = pd.merge(df_related_artists, df_all_tracks, on='related_id')
df_related_artists.head()

In [ ]:
#Reorganise the dataframe columns
solution = df_related_artists[['artist_name', 'artist_id', 'artist_genres', 'artist_popularity', 'artist_followers', 
                              'related_name', 'related_id', 'related_genres', 'related_popularity', 'related_followers', 
                              'related_href', 'related_track_name', 'related_uri']]
solution.head(30)

---

### Bonus track!!!

You can publish your own Playlist with [`/playlists`](https://developer.spotify.com/documentation/web-api/reference/#/operations/create-playlist) and [`/tracks`](https://developer.spotify.com/documentation/web-api/reference/#/operations/add-tracks-to-playlist).

![I quit](https://media.giphy.com/media/gui67fZ3xIneM/giphy.gif)

In [ ]:
# Bonus

#Instrucciones para crear un archivo donde almacenar la id de Spotify sin tener que borrarla

# Install dotenv lib: conda install -c conda-forge python-dotenv

# En git.bash puedo crear un archivo utilizando:  touch .env  (el archivo se llama '.env'). Para hacer esto hay que estar
# en la misma carpeta donde está el archivo

# Para abrir el archivo en gitbash utilizar: nano .env. Abajo aparecen las instrucciones para salir.

# Hay que añadir el '.env' al archivo '.gitignore' para que no se suba al repo. Para comprobar si está en el archivo
# utilizar el comunado: 'cat .gitignore' desde gitbash y desde el directorio donde está el archivo ''.gitignore'

#Incluir el import: 'from dotenv import dotenv_values'

In [ ]:
import base64
#import requests

# Your client ID and client secret
client_id = config.get('CLIENT_ID')   # CLIENT ID
client_secret = config['CLIENT_SECRET']   # CLIENT SECRET
user_id = "carloscsv"


# Construct the Basic Authorization header
auth_header = base64.b64encode(f'{client_id}:{client_secret}'.encode('utf-8')).decode('utf-8')

# Define the token request parameters
auth_options = {
    'url': 'https://accounts.spotify.com/api/token',
    'headers': {
        'Authorization': f'Basic {auth_header}'
    },
    'data': {
        'grant_type': 'client_credentials'
    }
}

# Send a POST request to obtain the access token
response = requests.post(**auth_options)

if response.status_code == 200:
    # Extract the access token from the response
    token = response.json().get('access_token')

    # Define the API request parameters with the access token
    api_url = f'https://api.spotify.com/v1/users/{user_id}'
    api_headers = {
        'Authorization': f'Bearer {token}'
    }

    # Send a GET request to the Spotify Web API
    api_response = requests.get(api_url, headers=api_headers)

    if api_response.status_code == 200:
        # Print the response from the Spotify API
        print(api_response.json())
    else:
        print(f'Failed to fetch data from the Spotify API: {api_response.status_code}')
else:
    print(f'Failed to obtain the access token: {response.status_code}')

In [ ]:
import webbrowser
from urllib.parse import urlparse, parse_qs

In [ ]:
redirect_uri = "https://example.com/callback"
scopes = "playlist-modify-public"

In [ ]:
authorization_url = f"https://accounts.spotify.com/authorize?client_id={client_id}&response_type=token&redirect_uri={redirect_uri}&scope={scopes}"
#Con esto se abre el navegador utilizando la url anterior
webbrowser.open(authorization_url)

In [ ]:
url = 'https://example.com/callback#access_token=BQDmv5RX3_rrjwBUQX55m48PyPEN4N8vCCudgvDndzDy1dQYMzE6LaUmsuozPzLc3m-eLOzHCXTGvfQ9I_m5bP4Vi69grqThKfQNbJf3h_MG_k70GsR0jnt55Dp5J_4NlcxJ1JGlzvapzmXTjdowY0hhU6Fb4Bwlt7BBoHcX0DBKFVA_Gv5VNQQGt4xd56sp7UT6PBKjbA&token_type=Bearer&expires_in=3600'
code = url.replace("https://example.com/callback#access_token=",'').split('&state')[0]
code

In [ ]:
# Define a function to extract the access token from the URL
def get_access_token():
    auth_url = input('Enter the URL after being redirected: ')
    parsed_url = urlparse(auth_url)
    fragment = parsed_url.fragment
    query_parameters = parse_qs(fragment)
    access_token = query_parameters.get('access_token', [None])[0]
    return access_token
# Obtain the access token
access_token = get_access_token()

In [ ]:
# Replace these variables with your own values
#access_token = “your_access_token_here”
user_id = 'carloscsv' # “your_user_id_here”
playlist_name = input('Enter a playlist name:')
playlist_description = f'Playlist para realizar practicas {playlist_name}'
# Define the API endpoint and headers
api_endpoint = f'https://api.spotify.com/v1/users/{user_id}/playlists'
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Create the playlist
playlist_data = {
    'name': playlist_name,
    'description': playlist_description,
    'public': True  # You can change the visibility as needed
}

In [ ]:
def add_tracks_to_playlist(access_token, playlist_id, track_uris):
    api_endpoint = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    data = {
        'uris': track_uris
    }
    response = requests.post(api_endpoint, headers=headers, json=data)
    if response.status_code == 201:
        print('Tracks added to the playlist successfully.')
    else:
        print('Failed to add tracks to the playlist.')

In [ ]:
add_tracks_to_playlist(access_token, playlist_id, track_uris)

### CÓDIGO RODRIGO

In [ ]:
import base64

def callback():
    redirect_uri='https://example.com/callback'
    auth_options = {
        'url': 'https://accounts.spotify.com/api/token',
        'data': {
            'code': code,
            'redirect_uri': redirect_uri,
            'grant_type': 'authorization_code'
        },
        'headers': {
            'Content-Type': 'application/x-www-form-urlencoded',
            'Authorization': 'Basic ' + base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode()
        }
    }

    response = requests.post(auth_options['url'], data=auth_options['data'], headers=auth_options['headers'])
    return response.json()

token_res = callback()
token_res

---

You can always try with the [wrapper](https://github.com/plamere/spotipy)!!!

![I quit](https://media.giphy.com/media/3oFzlXPvXYZ4q8VQOs/giphy.gif)

---